In [ ]:
import requests#Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy#authentication
import spotipy.util as util#authentication
from spotipy.oauth2 import SpotifyClientCredentials#authentication

In [ ]:
#Credentials
cid = '049ade7215e54c63a2b628f3784dc407'
secret = 'b86bedbebf53486b970e92b2cb7b612c'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [ ]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
def get_album_track(url,artist_uri):
    all_albums = []
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
        
    response = resp.json()
    
    album_limit = (response['limit'])-1
    
    for x in range(album_limit):
        try:
            track_uri = response['items'][x]['uri'].split(':')
            track_name = response['items'][x]['name'].replace('[','').replace(']','')
            print(track_uri,track_name)
            all_albums.append({'track_uri':track_uri[2],'track_name':track_name})
        except IndexError as error:
            continue #Natural part of the process. The length of response is sometimes less than limit
    try:
        if (url is not None):
            get_albums(response['next'])
        else:
            return
    except:
        return all_albums

In [ ]:
i = 0
track_list=[]

start = time.time()
with open('Data/artist_album_uri.csv') as csvfile:
    file = csv.reader(csvfile, delimiter='|')
    next(file)
    head = [next(file) for x in range(50)] #for limiting amount of rows read
    
    for row in head:
        artist_uri = row[0]
        artist_name = row[1]
        album_uri = row[2]
        print(i,artist_uri,artist_name,album_uri)
        albumtracks = get_album_track('https://api.spotify.com/v1/albums/{id}/tracks?limit=14'.format(id=album_uri),artist_uri)#list of album uris
        for track in albumtracks:
            track_list.append({'Artist URI':artist_uri,'Artist Name':artist_name,'Album URI':album_uri
                               ,'track_uri':track['track_uri'],'track_name':track['track_name']})#output album_uri
        
        i+=1
        
stop = time.time()
duration = (stop - start)/60

In [ ]:
#Duration of script
str(duration)+' minutes'

In [ ]:
df = pd.DataFrame(track_list)
df
#Test whether there are duplicates for Album URI
df_dropduplicates = df.drop_duplicates(['Artist URI', 'track_name'])
df_dropduplicates
df_dropduplicates.to_csv('Data/artist_album_track_uri.csv',sep='|',index=False)

In [ ]:
track_list